# SQL Tutorial 3: Joins

-   Let's use a database of experimental plate assays

In [1]:
%load_ext sql
%config SqlMagic.autolimit = 0
%config SqlMagic.displaylimit = 0
%sql sqlite:///data/assays.db

Connecting to 'sqlite:///data/assays.db'

-   Has six tables, but we'll start with these two
    -   `cid`: column ID (SQL lets use use numeric column IDs in some places, but please don't)
    -   `name`: column name
    -   `type`: data type (examples below are `INTEGER` and `TEXT`)
    -   `notnull`: are `null` values forbidden?
    -   `dflt_value`: default value for this column (we'll worry about this later)
    -   `pk`: is this the primary key for the table? (a major topic of this lesson)

**department**

| cid |   name   | type | notnull | dflt_value | pk |
|-----|----------|------|---------|------------|----|
| 0   | ident    | TEXT | 1       |            | 0  |
| 1   | name     | TEXT | 1       |            | 0  |
| 2   | building | TEXT | 1       |            | 0  |

**staff**

| cid |   name   |  type   | notnull | dflt_value | pk |
|-----|----------|---------|---------|------------|----|
| 0   | ident    | INTEGER | 0       |            | 1  |
| 1   | personal | TEXT    | 1       |            | 0  |
| 2   | family   | TEXT    | 1       |            | 0  |
| 3   | dept     | TEXT    | 0       |            | 0  |

-   What do these tables contain?

In [2]:
%%sql
select * from department;

Running query in 'sqlite:///data/assays.db'

ident,name,building
gen,Genetics,Chesson
hist,Histology,Fashet Extension
mb,Molecular Biology,Chesson


In [3]:
%%sql
select * from staff;

Running query in 'sqlite:///data/assays.db'

ident,personal,family,dept
1,Miraan,Sagar,None
2,Rohan,Chokshi,mb
3,Riya,Bahri,hist
4,Badal,Sura,gen
5,Rohan,Dara,mb
6,Sara,Venkatesh,mb
7,Ela,Mangal,mb
8,Zeeshan,Varghese,mb
9,Fateh,Khatri,None
10,Dhruv,Bhardwaj,None


-   What building does each person work in?

In [4]:
%%sql
select
    staff.personal,
    staff.family,
    department.name
from staff inner join department
on staff.dept = department.ident;

Running query in 'sqlite:///data/assays.db'

personal,family,name
Rohan,Chokshi,Molecular Biology
Riya,Bahri,Histology
Badal,Sura,Genetics
Rohan,Dara,Molecular Biology
Sara,Venkatesh,Molecular Biology
Ela,Mangal,Molecular Biology
Zeeshan,Varghese,Molecular Biology


-   A *join* combines all rows from one table with all rows from another
    -   *inner join* only keeps the rows where the `on` condition is satisfied
-   Use `table.column` syntax to avoid ambiguity
    -   Both `staff` and `department` have an `ident` column
-   Staff with no department are not included in the results because `null` doesn't match a department ID
-   Use a *left outer join* to keep all rows from the left table even if there aren't matches
    -   Fill missing values with `null`

In [5]:
%%sql
select
    staff.personal,
    staff.family,
    department.name
from staff left outer join department
on staff.dept = department.ident;

Running query in 'sqlite:///data/assays.db'

personal,family,name
Miraan,Sagar,None
Rohan,Chokshi,Molecular Biology
Riya,Bahri,Histology
Badal,Sura,Genetics
Rohan,Dara,Molecular Biology
Sara,Venkatesh,Molecular Biology
Ela,Mangal,Molecular Biology
Zeeshan,Varghese,Molecular Biology
Fateh,Khatri,None
Dhruv,Bhardwaj,None


>   A *right outer join* keeps all rows from the right table, even if there aren't matches.
>   SQLite doesn't bother to implement it, since you can achieve the same effect by swapping table order.
>   It *does* implement *full outer join*, which keeps unmatched rows from both sides (again, filling with nulls as needed).

-   Let's look at two more tables: `experiment` records the experiments that have been done, and `performed` records who has done which

**experiment**

| cid |  name   |  type   | notnull | dflt_value | pk |
|-----|---------|---------|---------|------------|----|
| 0   | ident   | INTEGER | 0       |            | 1  |
| 1   | kind    | TEXT    | 1       |            | 0  |
| 2   | started | TEXT    | 1       |            | 0  |
| 3   | ended   | TEXT    | 0       |            | 0  |

**performed**

| cid |    name    |  type   | notnull | dflt_value | pk |
|-----|------------|---------|---------|------------|----|
| 0   | staff      | INTEGER | 1       |            | 0  |
| 1   | experiment | INTEGER | 1       |            | 0  |

-   `performed` is sometimes called a *join table* because it's only purpose is to connect two other tables.
-   Why is it needed?
-   The relationship between `department` and `staff` is *one-to-many*
    -   Represent this by storing a *foreign key* in `staff` that refers to a *primary key* in `department`
-   The relationship between `staff` and `experiment` is *many-to-many*
    -   One person might do many experiments
    -   Each experiment might be done by many people
    -   So store each (person, experiment) pair in `performed`
-   There are a lot of experiments, so let's be selective

In [6]:
%%sql
select *
from staff inner join performed inner join experiment
on staff.ident = performed.staff and performed.experiment = experiment.ident
where staff.ident in (1, 2);

Running query in 'sqlite:///data/assays.db'

ident,personal,family,dept,staff,experiment,ident_1,kind,started,ended
1,Miraan,Sagar,None,1,3,3,trial,2023-10-10,2023-10-11
1,Miraan,Sagar,None,1,19,19,trial,2023-12-16,2023-12-17
2,Rohan,Chokshi,mb,2,28,28,trial,2023-05-14,2023-05-15
1,Miraan,Sagar,None,1,28,28,trial,2023-05-14,2023-05-15
1,Miraan,Sagar,None,1,29,29,calibration,2023-10-11,2023-10-11
2,Rohan,Chokshi,mb,2,32,32,trial,2023-06-28,2023-06-29
1,Miraan,Sagar,None,1,37,37,trial,2023-12-02,2023-12-04
1,Miraan,Sagar,None,1,39,39,calibration,2024-01-21,2024-01-21
1,Miraan,Sagar,None,1,42,42,trial,2023-06-04,2023-06-06
1,Miraan,Sagar,None,1,46,46,trial,2023-06-17,2023-06-19


-   First step: database combines `staff` with `performed` by matching primary key in the former to foreign key in the latter
    -   We'll only show the first few rows of the result

In [7]:
%%sql
select *
from staff inner join performed
on staff.ident = performed.staff
limit 5;

Running query in 'sqlite:///data/assays.db'

ident,personal,family,dept,staff,experiment
5,Rohan,Dara,mb,5,1
5,Rohan,Dara,mb,5,2
7,Ela,Mangal,mb,7,2
1,Miraan,Sagar,None,1,3
8,Zeeshan,Varghese,mb,8,4


-   Second step: database combines this temporary table with `experiment` by matching keys
    -   Again, only show a few rows

In [8]:
%%sql
select *
from staff inner join performed inner join experiment
on staff.ident = performed.staff and performed.experiment = experiment.ident
limit 5;

Running query in 'sqlite:///data/assays.db'

ident,personal,family,dept,staff,experiment,ident_1,kind,started,ended
5,Rohan,Dara,mb,5,1,1,trial,2024-01-02,2024-01-03
5,Rohan,Dara,mb,5,2,2,trial,2023-03-12,2023-03-13
7,Ela,Mangal,mb,7,2,2,trial,2023-03-12,2023-03-13
1,Miraan,Sagar,None,1,3,3,trial,2023-10-10,2023-10-11
8,Zeeshan,Varghese,mb,8,4,4,calibration,2024-01-19,2024-01-19


-   Third step: database filters the result using `in` and a list of specific staff IDs
    -   `staff.ident in (1, 2)` is the same as `(staff.ident = 1) or (staff.ident = 2)`
    -   Again, only show a few rows (we saw the full output earlier)

In [9]:
%%sql
select *
from staff inner join performed inner join experiment
on staff.ident = performed.staff and performed.experiment = experiment.ident
where staff.ident in (1, 2)
limit 5;

Running query in 'sqlite:///data/assays.db'

ident,personal,family,dept,staff,experiment,ident_1,kind,started,ended
1,Miraan,Sagar,None,1,3,3,trial,2023-10-10,2023-10-11
1,Miraan,Sagar,None,1,19,19,trial,2023-12-16,2023-12-17
2,Rohan,Chokshi,mb,2,28,28,trial,2023-05-14,2023-05-15
1,Miraan,Sagar,None,1,28,28,trial,2023-05-14,2023-05-15
1,Miraan,Sagar,None,1,29,29,calibration,2023-10-11,2023-10-11


-   Notice that the `ident` column from `experiment` has been named `ident_1` in the output
-   Better practice to slim down the columns (e.g., remove the columns from the join table) and rename any duplicates
-   Just for fun, we will add `offset 5` to look at rows *after* row 5 and order by start date
    -   Note: sort *then* slice

In [10]:
%%sql
select
    staff.personal,
    staff.family,
    experiment.ident as experiment_id,
    experiment.kind,
    experiment.started,
    experiment.ended
from staff inner join performed inner join experiment
on staff.ident = performed.staff and performed.experiment = experiment.ident
where staff.ident in (1, 2)
order by experiment.started asc
limit 5 offset 5;

Running query in 'sqlite:///data/assays.db'

personal,family,experiment_id,kind,started,ended
Rohan,Chokshi,50,trial,2023-10-09,2023-10-10
Miraan,Sagar,3,trial,2023-10-10,2023-10-11
Miraan,Sagar,29,calibration,2023-10-11,2023-10-11
Miraan,Sagar,37,trial,2023-12-02,2023-12-04
Miraan,Sagar,19,trial,2023-12-16,2023-12-17


-   How many experiments of each kind has each person been involved in?

In [11]:
%%sql
select
    staff.personal,
    staff.family,
    experiment.kind,
    count(*) as num_experiment_kind
from staff inner join performed inner join experiment
on staff.ident = performed.staff and performed.experiment = experiment.ident
group by staff.ident, experiment.kind
order by staff.family, staff.personal, experiment.kind;

Running query in 'sqlite:///data/assays.db'

personal,family,kind,num_experiment_kind
Riya,Bahri,calibration,4
Riya,Bahri,trial,4
Dhruv,Bhardwaj,calibration,3
Dhruv,Bhardwaj,trial,2
Rohan,Chokshi,trial,3
Rohan,Dara,calibration,1
Rohan,Dara,trial,2
Fateh,Khatri,calibration,3
Fateh,Khatri,trial,5
Ela,Mangal,calibration,3


-   How many *kinds* of experiments was each person involved in?

In [12]:
%%sql
select
    staff.personal,
    staff.family,
    count (distinct experiment.kind) as num_kind
from staff inner join performed inner join experiment
on staff.ident = performed.staff and performed.experiment = experiment.ident
group by staff.ident
order by num_kind, staff.family, staff.personal;

Running query in 'sqlite:///data/assays.db'

personal,family,num_kind
Rohan,Chokshi,1
Riya,Bahri,2
Dhruv,Bhardwaj,2
Rohan,Dara,2
Fateh,Khatri,2
Ela,Mangal,2
Miraan,Sagar,2
Badal,Sura,2
Zeeshan,Varghese,2
Sara,Venkatesh,2
